# 带电粒子径迹重建: 循迹

带电粒子径迹的重建是粒子、核与粒子天体物理实验中最重要、也是最复杂和最消耗 CPU 的事例处理链条。径迹重建精度对物理测量精度有直接而显著的影响。同时，未来粒子物理对撞机亮度大幅增加所带来的大量堆积背景和数据率对径迹重建的精度、效率和速度提出了巨大挑战。

本教程利用缪子反常磁矩实验的合成径迹数据，基于图神经网络中的边分类任务来实现径迹重建中的径迹寻找任务。

## 技术路径
解决这个问题的具体流程如下：

创建数据集

模型构建

损失函数

模型训练

### 网络架构

![LorentzNet](./images/TrackFinding.PNG)

In [1]:
#通用包
import numpy as np
import os, gc
import sys, warnings
from datetime import datetime
import argparse, json, time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#mindspore相关库
import mindspore
import mindspore.nn as nn  
from mindspore import Tensor
import mindspore.ops as ops  

#louvain相关库
import networkx as nx
import networkx.algorithms as algos

#自有库
import src.utils as utils
import src.model as model
import src.dataset as ds
from src.train import train_loop, test_loop, forward_fn

In [2]:
#定义训练必要参数
config = {
            'exp_name'      : 'TrackFinding',     
            'layer'         : 128,
            'batch_size'    : 32,
            'gpu'           : 'CPU',
            'seed_num'      : 827,
            'num_of_epochs' : 10,
            'lr'            : 0.0001,
            'root_name'     : './data/30996_n2000',
            'Ndata'         : 3200,
            'graph_mode'    : 1, 
}

#初始化log目录、结果存储和gpu设备
logdir = utils.initial(config)
#将训练参数进行保存，以备复现
with open(f"{logdir}/config.json",'w') as config_file:
    config_file.write( json.dumps(config, indent=4) )

the compilation framework is set to dynamic graph mode
[initial seed] >>> seed num set to 827, PID:960817
use device CPU


### 创建数据集
缪子反常磁矩实验合成径迹数据集，以CERN ROOT格式存储

In [3]:
#建立数据集
src, dst, node_feat, edge_feat, node_match, node_track, edge_label = ds.root_to_data(config['root_name'], mode='coo', Ndata=config['Ndata'])
train_set, test_set, apply_set = utils.build_dataset(src, dst, node_feat, edge_feat, edge_label)

_, train_loader = utils.batch(train_set, bs=config['batch_size'], shuffle=True, drop_last=True)
_, test_loader = utils.batch(test_set, bs=config['batch_size'], shuffle=False, drop_last=True)
_, apply_loader = utils.batch(apply_set, bs=config['batch_size'], shuffle=False, drop_last=True)

### 模型构建：
参数定义：

embed_size: 嵌入维度和隐藏层维度

In [4]:
#定义模型、优化器、损失函数
net = model.Finding(embed_size=config['layer'])
params_size = sum(p.numel() for p in net.trainable_params())
print(f"Model parameters size: {params_size}")

Model parameters size: 1043463


### 损失函数
定义损失函数和优化器

In [5]:
criterion = nn.BCEWithLogitsLoss(reduction='sum')
optimizer = nn.Adam(net.trainable_params(), learning_rate=config['lr'], weight_decay=0.01)
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

### 模型训练
训练模型并保存检查点，利用Mindspore Insight记录训练过程

In [6]:
print('Train')
for t in range(config['num_of_epochs']):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(net, train_loader, criterion, grad_fn, optimizer, t, 'TrackFinding_CPU')
    test_loop(net, apply_loader, criterion)

print('Test')
test_loop(net, test_loader, criterion)
print("Done!")

Train
Epoch 1
-------------------------------
 [**************************************************]100/100 - used 122.8s / left 0.0s / loss 0.4136839674 / acc 0.8074 Valid: 
 Accuracy: 81.6%, Avg loss: 0.242488 

Epoch 2
-------------------------------
 [**************************************************]100/100 - used 119.2s / left 0.0s / loss 0.2017527625 / acc 0.8938 Valid: 
 Accuracy: 93.2%, Avg loss: 0.179330 

Epoch 3
-------------------------------
 [**************************************************]100/100 - used 125.1s / left 0.0s / loss 0.1617302472 / acc 0.9339 Valid: 
 Accuracy: 94.1%, Avg loss: 0.151917 

Epoch 4
-------------------------------
 [**************************************************]100/100 - used 120.3s / left 0.0s / loss 0.1427512181 / acc 0.9430 Valid: 
 Accuracy: 94.6%, Avg loss: 0.142756 

Epoch 5
-------------------------------
 [**************************************************]100/100 - used 124.3s / left 0.0s / loss 0.1289977705 / acc 0.9497 Valid:

AttributeError: module 'src.model' has no attribute 'set_train'